## 태양광 발전량 예측 프로젝트
- 목적 : 오산, 광명 2곳의 태양광 발전량 데이터를 바탕으로, 발전량 예측 모델 구축 (*기존 예측오차율 7.2% 이하 달성) 
- Data Source 1 : 태양광 발전량 모니터링 시스템 https://www.solarcube.co.kr/SrMain/SM010.aspx  (id/pw = sclossolar) 
  - cf. solar cube site에서 크롤링 대상 페이지  : 통계보고서 > 시간대별 리포트 (생산량/kwh) 
- Data Source 2 : 기상관측 데이터 API 
- Data Source 3 : 날짜/시간대별 태양위치  

#### 변수설명 

1) 고도변화 : 해당시간의 last - first 고도차이 (*해당시간의 평균고도차로 대체가능)
2) 방위각 : 태양의 수평각도 
3) 태양위치 데이터 : 기존 코드에서 날짜와 위,경도 데이터를 대입하여 산출 
4) 변수선택 기준 : 자의적 판단 


#### 코드설명 
1) Class ~~ -> 발전량 웹사이트 데이터 크롤링하여 가공하는 부분 
2) Class forecast -> 기상청 api로 부터 강수량/운량 등 데이터를 가져오는 부분 
3) Class weather -> 기상관측 데이터 api 가져오기 

4) def solar -> 태양위치 구하기 
5) def site_sum -> 오산&광명 데이터를 합치기 


#### 기타 
1) 운량 : 관측치 10단계를 -> 예보 4단계로 변환 
2) 기상자료 개방 포털 : 관측소가 수원밖에 없음 

In [6]:
## 사용할 패키지 설치 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time, math, datetime, requests, json

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# 소수점 2자리에서 반올림, 천단위 쉼표
pd.options.display.float_format = '{:,.2f}'.format

### Part I. Web site로 부터 태양광 발전량 데이터 가져오기 
  - url : https://www.solarcube.co.kr/SrMain/SM010.aspx (id/pw = sclossolar)
  - chromedriver 다운&설치 : https://chromedriver.chromium.org/downloads ("C:\chromedriver\chromedriver.exe)"

In [7]:
## 발전량 데이터셋을 만드는 함수 output을 다음과 같이 정의한다. 
class output:

    def get_output(self, site, year, month, day):
    
        ## 키보드 설정

        self.ENTER = '/ue007'
        self.BACKSPACE = '/ue003'
        self.LEFT = '/ue012'
        self.UP = '/ue013'
        self.RIGHT ='/ue014'
        self.DOWN = '/ue015'

        ## 크롬을 이용해서 사이트 불러오기

        self.url = 'https://www.solarcube.co.kr/SrMain/SM010.aspx'
        self.driver = webdriver.Chrome('c:/chromedriver/chromedriver.exe')
        self.driver.get(self.url)

        ## 사이트에서 원하는 정보 가져오기

        self.driver.find_element_by_xpath('//*[@id="Txt_1"]').send_keys('sclossolar')
        time.sleep(1.0)
        self.driver.find_element_by_xpath('//*[@id="Txt_2"]').send_keys('sclossolar')
        time.sleep(1.0)
        self.driver.find_element_by_xpath('//*[@id="imageField"]').click()
        time.sleep(1.0)
        self.driver.find_element_by_xpath('//*[@id="SrTop_Panel1"]/div[2]/ul/li[5]/a').click()
        time.sleep(1.0)
        self.driver.find_element_by_xpath('//*[@id="select2-SrTop_cbo_plant_d1-container"]').click()
        time.sleep(1.0)
        self.web1 = self.driver.find_element_by_xpath('/html/body/span/span/span[1]/input')
        self.web1.send_keys('%s' % site)
        time.sleep(1.0)
        self.web1.send_keys(Keys.ENTER)
        time.sleep(1.0)
        self.web2 = self.driver.find_element_by_xpath('//*[@id="txt_Calendar"]')
        self.web2.click()
        time.sleep(1.0)
        
        for i in range(0,10):
            self.web2.send_keys(Keys.RIGHT)
        
        for i in range(0,10):
            self.web2.send_keys(Keys.BACKSPACE)
        
        self.web2.send_keys('%s-%s-%s' % (year, month, day))
        time.sleep(0.5)
        self.driver.find_element_by_xpath('//*[@id="submitbtn"]').click()
        time.sleep(1.0)

        ## 전체 내용 불러오기

        self.html = BeautifulSoup(self.driver.page_source, 'html.parser') # html 전체 내용 불러오기
        self.data = self.html.select('div.sub_contents > div.s_cont > table.sub_com_table > tbody > tr > td') # 발전실적 테이블 추출

        ## 크롬창 닫기

        self.driver.close()

        # 발전량 데이터 전처리 및 데이터프레임 만들기

        self.data_text = []

        for i in range(len(self.data)):
            self.a = str(self.data[i]).replace('<td>', '')
            self.b = self.a.replace('</td>', '')
            self.c = self.b.replace('<span class="f_w600">', '')
            self.d = self.c.replace('</span>', '')
            self.e = self.d.replace('<span class="align_C">', '')
            self.f = self.e.replace('<span class="align_R">', '')
            self.data_text.append(self.f)

        self.data_num = len(self.data_text) // 4

        self.data_array = np.array(self.data_text) # 리스트를 넘파이 어레이화
        self.data_array = self.data_array.reshape(int(self.data_num), 4) # nx4 행렬로 변환
        self.output_yesterday = pd.DataFrame(self.data_array, columns = ['site', 'date', 'output', 'radiation']) # 행렬을 데이터프레임으로 변환
        
        return self.output_yesterday

### Part II. D+1일의 기상예보 데이터 가져오기  
  - 공공데이터포털(www.data.go.kr)의 ‘기상청_단기예보 ((구)_동네예보) 조회서비스’ API 이용 (오산, 광명에서 가장 근접한 위치의 동네 예보 데이터) 
    - https://www.data.go.kr/data/15084084/openapi.do 
  - API 정보 (인증키, 일시, 사이트 위치 등) 입력하여 데이터 요청 
  - 발전량 예측에 필요한 데이터 (온도, 풍속, 습도, 운량 등) 추출 및 정리

In [19]:
## class forcast를 다음과 같이 정의한다. 
class forecast:
    
    # 기상 데이터 불러오기 함수 get_forecast() 정의

    def get_forecast(self, base_date, rows, base_time, nx, ny):
        self.servicekey = 'C5XWwKpv9QPJ8%2FX1nDI7qY2ZgVrDp9r4fzIXYCvjfj5uZ6MbPHY3SxTJkzrOpzDUrYW5j4%2FEJTklpG3JZzhigg%3D%3D'
        self.str = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?ServiceKey={servicekey}&pageNo={page}&numOfRows={rows}&dataType={data_type}&base_date={base_date}&base_time={base_time}&nx={nx}&ny={ny}'
        self.url = self.str.format(servicekey=self.servicekey, page='1', rows=rows, data_type='json', base_date=base_date, base_time=base_time, nx=nx, ny=ny)
        self.resp = requests.get(self.url)
        self.data = json.loads(self.resp.text)
        return self.data

    # 불러온 데이터 정리하기 함수 make_df() 정의

    def make_forecast_df(self, data):
        self.data_items = data['response']['body']['items']['item']
        
        self.df_data = pd.DataFrame(self.data_items)
        self.df_data_sort = self.df_data.sort_values(by = 'category')

        # 예보항목 = ['TMP(온도)', 'WSD(풍속)', 'REH(습도)', 'SKY(운량)']

        self.TMP = self.df_data_sort.loc[self.df_data_sort.category == 'TMP', :]
        self.TMP= self.TMP.sort_index()
        self.TMP.columns = ['announce_date', 'announce_time', 'announce', 'forecast_date', 'forecast_time', 'temperature', 'x', 'y']
        #self.TMP = self.TMP.set_index('forecast_time')
        self.TMP = self.TMP.drop(['announce', 'x', 'y'], axis = 1)

        self.PCP = self.df_data_sort.loc[self.df_data_sort.category == 'PCP', :]
        self.PCP = self.PCP.sort_index()
        self.PCP = self.PCP.drop(['baseDate', 'baseTime', 'category', 'nx', 'ny'], axis = 1)
        self.PCP.columns = ['forecast_date', 'forecast_time', 'rainfall']
        self.PCP = self.PCP.set_index('forecast_time')
        self.PCP['rainfall'].replace('강수없음', '0', inplace = True)
        self.PCP['rainfall'].replace('1mm 미만', '0', inplace = True)
        self.PCP['rainfall'].replace('30~50mm', '40', inplace = True)
        self.PCP['rainfall'].replace('50mm 이상', '50', inplace = True)
        self.PCP['rainfall'] = self.PCP['rainfall'].str.replace('mm', '')
        self.PCP['rainfall'].astype('float64')
        self.forecast = pd.merge(self.TMP, self.PCP, how = 'left', on = ['forecast_time', 'forecast_date'])

        self.WSD = self.df_data_sort.loc[self.df_data_sort.category == 'WSD', :]
        self.WSD = self.WSD.sort_index()
        self.WSD = self.WSD.drop(['baseDate', 'baseTime', 'category', 'nx', 'ny'], axis = 1)
        self.WSD.columns = ['forecast_date', 'forecast_time', 'windvelocity']
        #self.WSD = self.WSD.set_index('forecast_time')
        self.forecast = pd.merge(self.forecast, self.WSD, how = 'left', on = ['forecast_time', 'forecast_date'])

        self.REH = self.df_data_sort.loc[self.df_data_sort.category == 'REH', :]
        self.REH = self.REH.sort_index()
        self.REH = self.REH.drop(['baseDate', 'baseTime', 'category', 'nx', 'ny'], axis = 1)
        self.REH.columns = ['forecast_date', 'forecast_time', 'humidity']
        #self.REH = self.REH.set_index('forecast_time')
        self.forecast = pd.merge(self.forecast, self.REH, how = 'left', on = ['forecast_time', 'forecast_date'])

        self.SNO = self.df_data_sort.loc[self.df_data_sort.category == 'SNO', :]
        self.SNO = self.SNO.sort_index()
        self.SNO = self.SNO.drop(['baseDate', 'baseTime', 'category', 'nx', 'ny'], axis = 1)
        self.SNO.columns = ['forecast_date', 'forecast_time', 'snowfall']
        self.SNO = self.SNO.set_index('forecast_time')
        self.SNO['snowfall'].replace('적설없음', '0', inplace = True)
        self.SNO['snowfall'].replace('1cm 미만', '0', inplace = True)
        self.SNO['snowfall'].replace('1cm미만', '0', inplace = True)
        self.SNO['snowfall'].replace('5cm 이상', '5', inplace = True)
        self.SNO['snowfall'].replace('5cm이상', '5', inplace = True)
        self.SNO['snowfall'] = self.SNO['snowfall'].str.replace('mm', '')
        self.SNO['snowfall'].astype('float64')
        self.forecast = pd.merge(self.forecast, self.SNO, how = 'left', on = ['forecast_time', 'forecast_date'])

        self.SKY = self.df_data_sort.loc[self.df_data_sort.category == 'SKY', :]
        self.SKY = self.SKY.sort_index()
        self.SKY = self.SKY.drop(['baseDate', 'baseTime', 'category', 'nx', 'ny'], axis = 1)
        self.SKY.columns = ['forecast_date', 'forecast_time', 'sky']
        #self.SKY = self.SKY.set_index('forecast_time')
        self.forecast = pd.merge(self.forecast, self.SKY, how = 'left', on = ['forecast_time', 'forecast_date'])
    
        return self.forecast

### Part III. D-1일의 기상관측 데이터 가져오기 

  - 공공데이터포털(www.data.go.kr)의 ‘기상청_지상(종관, ASOS) 시간자료 조회서비스’ API 이용
    - https://www.data.go.kr/data/15057210/openapi.do
  - 오산, 광명과 근접한 수원 관측소 데이터를 가져옴
  - API 정보(인증키, 일시, 관측소 번호 등) 입력하여 데이터 요청
  - 획득한 데이터 중 발전량 예측에 필요한 데이터 (온도, 풍속, 습도, 운량 등) 추출 및 정리

In [23]:
## class weather를 다음과 같이 정의한다. 
class weather:
    
    # 기상 데이터 불러오기 함수 get_forecast() 정의

    def get_weather(self, rows, start_date, start_hour, end_date, end_hour, location):
        self.servicekey = 'C5XWwKpv9QPJ8%2FX1nDI7qY2ZgVrDp9r4fzIXYCvjfj5uZ6MbPHY3SxTJkzrOpzDUrYW5j4%2FEJTklpG3JZzhigg%3D%3D'
        self.str = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey={servicekey}&numOfRows={rows}&pageNo={page}&dataType={data_type}&dataCd=ASOS&dateCd=HR&stnIds={location}&endDt={end_date}&endHh={end_hour}&startHh={start_hour}&startDt={start_date}'
        self.url = self.str.format(servicekey=self.servicekey, page='1', rows=rows, data_type='json', start_date=start_date, start_hour=start_hour, end_date=end_date, end_hour=end_hour, location=location)
        self.resp = requests.get(self.url)
        self.data = json.loads(self.resp.text)
        
        return self.data

    # 불러온 데이터 정리하기 함수 make_weather_df() 정의
    
    def make_weather_df(self, data):
        self.data_items = data['response']['body']['items']['item']
        
        self.df_data = pd.DataFrame(self.data_items)
        self.weather_df = self.df_data.drop(['stnId', 'rnum', 'taQcflg', 'rnQcflg', 'wsQcflg', 'wd', 'wdQcflg', 'hmQcflg', 'pv', 'td', 'pa', 'paQcflg', 'ps', 'psQcflg', 'ss', 'ssQcflg', 'icsr', 'hr3Fhsc', 'dc10LmcsCa', 'clfmAbbrCd', 'lcsCh', 'vs', 'gndSttCd', 'dmstMtphNo', 'ts', 'tsQcflg', 'm005Te', 'm01Te', 'm02Te', 'm03Te'], axis = 1)
        self.weather_df.columns = ['date', 'location', 'temperature', 'rainfall', 'windvelocity', 'humidity', 'snowfall', 'sky']
        
        return self.weather_df